In [1]:
from konlpy.tag import Okt
import networkx as nx
import numpy as np
from itertools import combinations
from collections import Counter

In [2]:
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

In [3]:
okt = Okt()

In [4]:
def tokenize(text):
    # 형태소 분석을 통해 명사만 추출
    return [word for word in okt.nouns(text) if len(word) > 1]

In [5]:
def extract_keywords(text, num_keywords=10):
    okt = Okt()
    
    # 형태소 분석 및 명사 추출
    words = tokenize(text)
    
    # 단어들의 출현 빈도수 계산
    word_counts = Counter(words)
    
    # 그래프 생성
    graph = nx.Graph()
    
    # 단어 노드 추가
    for word, count in word_counts.items():
        if count > 1:  # 최소 출현 빈도 조건
            graph.add_node(word, count=count)
    
    # 단어의 연결을 위한 윈도우 크기 설정
    window_size = 4
    
    # 윈도우 안에서 단어 간의 연결 설정
    for i in range(len(words) - window_size + 1):
        window_words = words[i:i + window_size]
        for w1, w2 in combinations(window_words, 2):
            if graph.has_node(w1) and graph.has_node(w2):
                if graph.has_edge(w1, w2):
                    graph[w1][w2]['weight'] += 1
                else:
                    graph.add_edge(w1, w2, weight=1)
    
    # PageRank 계산
    rank = nx.pagerank(graph, weight='weight')
    
    # 상위 num_keywords개의 키워드 추출
    top_keywords = sorted(rank.items(), key=lambda x: x[1], reverse=True)[:num_keywords]
    
    return [keyword for keyword, _ in top_keywords]

In [26]:
def parse_text_file(file_path):
    result_dict = {}

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # 줄의 앞뒤 공백 제거
            line = line.strip()

            # C?? 형식의 코드와 나머지 문자열 분리
            if line:
                parts = line.split(' ', 1)  # 첫 번째 공백으로 나눔
                if len(parts) == 2:
                    key = parts[0]
                    values = parts[1].split(',')  # 쉼표와 공백으로 구분하여 리스트 생성
                    result_dict[key] = values

    return result_dict

In [27]:
def count_matches(keyword_list, reference_list):
    return sum(1 for keyword in keyword_list if keyword in reference_list)

In [6]:
directory_path = '/home/osung/data/korean/modu/json'
df = pd.read_csv(directory_path+'/combined_news.tsv', sep='\t')

In [7]:
len(df)

3611424

In [8]:
df.columns

Index(['id', 'topic', 'text'], dtype='object')

In [10]:
df_e = df[df.topic == '경제']

In [15]:
del df

In [16]:
len(df_e)

864427

In [17]:
documents = df_e.text

In [ ]:
doc_keywords = []

for doc in tqdm(documents) :
    keywords = extract_keywords(doc)
    doc_keywords.append(keywords)

 95%|████████████████████████████▌ | 824287/864427 [68:57:02<6:20:18,  1.76it/s]

In [19]:
doc_keywords

[['트리뷴', '컴퍼니', '미국', '달러', '광고', '뉴욕타임스', '언론사', '인수', '언론', '경영'],
 ['이전', '계획', '지방', '한국', '국립', '승인', '학원', '공사', '석유', '공공기관'],
 ['아파트', '가격', '강남', '재건축', '기사', '상승', '일부', '매물', '사무소', '거래'],
 ['공장', '가동', '생산', '대우', '부평', '젠트', '라세티', '수출', '중단', '불황'],
 ['피해자', '최씨', '아파트', '사기', '조합', '분양', '계약', '통해', '안양', '대림'],
 ['버섯', '재배', '허가', '목적', '구역', '개발', '관리', '다른', '해당', '지역'],
 ['부동산', '통계', '시장', '재건축', '아파트', '자료', '하락', '기사', '강남', '정보'],
 ['아파트', '보증금', '분양', '잔금', '판교', '판교신도시', '대출', '주가', '이자', '전세'],
 ['사업', '주택', '덕천', '재개발', '시행', '지구', '세대', '아파트', '건설', '안양시'],
 ['스와프', '통화', '한도', '미국', '무제한', '한국', '협정', '기간', '한미', '방안'],
 ['공사', '지역', '조기', '인천', '예산', '사업', '경우', '집행', '위해', '업체'],
 ['지식', '정보', '타운', '과천', '과천시', '사업', '도시', '산업', '개발', '주거'],
 ['구조조정', '국민은행', '점검', '내부', '통제', '위해', '임금', '노동자', '정규직', '은행'],
 ['인천', '전망', '자금', '악화', '전국', '경제', '성장', '지역', '크게', '기업'],
 ['기술', '중국', '쌍용차', '상하이차', '유출', '투자', '자본', '논란', '인수', '이디스'],
 ['사실', '달러', '정부

In [20]:
len(doc_keywords)

864427

In [22]:
df_e['keywords'] = doc_keywords

In [23]:
df_e[:10]

,id,topic,text,keywords
12,NIRW1900000001.27,경제,미 '미디어 공룡' 위기는 '언론악법'의 미래 서브프라임 모기지 사태로 촉발된 미국...,"[트리뷴, 컴퍼니, 미국, 달러, 광고, 뉴욕타임스, 언론사, 인수, 언론, 경영]"
13,NIRW1900000001.31,경제,한국석유공사 2012년까지 울산으로 이전 경기 안양시 평촌 도심 한복판에 자리한 한...,"[이전, 계획, 지방, 한국, 국립, 승인, 학원, 공사, 석유, 공공기관]"
26,NIRW1900000001.51,경제,"""강남집값 기지개? 부녀회서 세뱃돈 받았나"" ""정초부터 부동산을 띄우는 이유가 뭘까...","[아파트, 가격, 강남, 재건축, 기사, 상승, 일부, 매물, 사무소, 거래]"
84,NIRW1900000001.176,경제,"GM대우, 부평1공장 가동재개 GM대우 부평1공장이 다시 가동에 들어갔다. 경제 불...","[공장, 가동, 생산, 대우, 부평, 젠트, 라세티, 수출, 중단, 불황]"
86,NIRW1900000001.180,경제,"""남의 집 제집이라 속여 팔았는데 사기 아니라고?"" 안양 비산동 대림 아파트 사기 ...","[피해자, 최씨, 아파트, 사기, 조합, 분양, 계약, 통해, 안양, 대림]"
118,NIRW1900000001.234,경제,환경부는 사들이고 자치단체는 '개발허가' 충남 금산군이 환경부가 수변구역으로 지정한...,"[버섯, 재배, 허가, 목적, 구역, 개발, 관리, 다른, 해당, 지역]"
125,NIRW1900000001.241,경제,"강남 아파트 하락? 상승?오락가락 부동산 기사, 진짜로 믿으시나요 '집값 하락폭 다...","[부동산, 통계, 시장, 재건축, 아파트, 자료, 하락, 기사, 강남, 정보]"
142,NIRW1900000001.262,경제,"""판교는 '망교'... 빚 지고 봉됐다"" ""판교 로또? 다 옛날 얘기다. 지금은 판...","[아파트, 보증금, 분양, 잔금, 판교, 판교신도시, 대출, 주가, 이자, 전세]"
147,NIRW1900000001.273,경제,"안양시, 덕천지구주택재개발 시행 인가 안양시의 대표적인 노후 주택지역으로 철도소음과...","[사업, 주택, 덕천, 재개발, 시행, 지구, 세대, 아파트, 건설, 안양시]"
187,NIRW1900000001.336,경제,한미 통화스와프 한도 무제한 추진 현재 300억 달러로 묶여 있는 한미 통화스와프의...,"[스와프, 통화, 한도, 미국, 무제한, 한국, 협정, 기간, 한미, 방안]"


In [24]:
df_e.to_csv(directory_path+'/combined_news_E.tsv', sep='\t', index=False)

In [28]:
file_path = '/home/osung/Downloads/KSIC_keyword.txt'
KSIC_key = parse_text_file(file_path)

In [29]:
combined_list = []

for key in KSIC_key :
    combined_list.extend(KSIC_key[key])

In [30]:
# 각 행에 대해 일치하는 문자열의 개수를 세고 새로운 열에 저장
df_e['match_count'] = df_e['keywords'].apply(lambda x: count_matches(x, combined_list))

In [31]:
df = df_e.sort_values(by='match_count', ascending=False)

In [32]:
df[:10]

,id,topic,text,keywords,match_count
3489245,NWRW2100000003.7968,경제,세계 첫 극저온용 고망간강 개발… 스마트 고로 4기도 가동 포스코는 지난달 30일 ...,"[포스코, 고로, 제품, 스마트, 망간, 철강, 강판, 탱크, 자동차, 세계]",7
2030330,NPRW1900000070.29415,경제,"현대차그룹, 수소전기차 생산 확대…현대제철 '빛' 보나 현대자동차그룹이 수소전기차 ...","[생산, 자동차, 제철, 현대차, 수소, 전기차, 강판, 강재, 계획, 현대]",7
1514417,NPRW1900000006.1499,경제,"LG그룹, 전기차 부품 '세계 최강자' 향해 질주 LG그룹은 에너지와 에코, 헬스케...","[부품, 자동차, 전자, 분야, 전기차, 사업, 매출, 차량, 디스플레이, 배터리]",7
1777136,NPRW1900000056.32982,경제,車에 다가서는 삼성 vs IT분야 노크하는 현대차 “바퀴 달린 것은 냉장고 청소기 ...,"[사업, 부품, 삼성, 전자, 자동차, 차량, 전지, 디스플레이, 현대차, 확대]",7
1842950,NPRW1900000059.21894,경제,삼성디스플레이 덕보는 삼성전기·삼성SDI 삼성디스플레이가 올가을 출시될 애플 아이폰...,"[삼성, 전기, 디스플레이, 실적, 전자, 패널, 공급, 아이폰, 부품, 갤럭시]",7
855041,NLRW1900000122.866,경제,"거듭난 스마트 제철, 철강 미래 이끈다…포스코 4대 메가트렌드 주목 ""미래 4가지 ...","[포스코, 강판, 제품, 철강, 케이블, 프리미엄, 미래, 전기차, 가전, 스틸]",7
144498,NIRW1900000030.5985,경제,"한국화학연구원, 수소자동차 연료전지 핵심 재료 물질 국산화 성공 한국화학연구원은 박...","[기술, 개발, 물질, 공정, 자동차, 수소, 원료, 전지, 불소, 연료]",7
1739838,NPRW1900000055.15153,경제,"""정제마진 줄어드나"" 정유업계도 전전긍긍 ◆ 셰일가스의 공습 ◆ 정유업계는 석유화학...","[석유, 연료, 정유, 셰일가스, 정제, 나프타, 대체, 업계, 가스, 석유화학]",7
3203568,NWRW1900000039.11007,경제,포스코 “철의 새 시대 연다”···기가스틸 TV 광고로 주목 “기가스틸로 철의 새 ...,"[스틸, 포스코, 소재, 광고, 알루미늄, 철강, 이미지, 강판, 기업, 강도]",6
987072,NLRW1900000157.1699,경제,친환경 통한 지속성장 모멘텀 확보를 화석 연료 기반의 사회는 한정된 자원의 고갈에 ...,"[자동차, 연료, 엔진, 시스템, 기술, 수소, 개발, 부품, 센서, 연구개발]",6


In [33]:
df_e = df[df['match_count'] >= 1]

In [34]:
len(df_e)

279828

In [35]:
len(df)

864427

In [36]:
df_e.to_csv(directory_path+'/news_E_keyword.tsv', sep='\t', index=False)